In [1]:
import pandas as pd

In [24]:
test_unseen = pd.read_csv('../data/test_unseen.csv')
train = pd.read_csv('../data/train.csv')
val_seen = pd.read_csv('../data/val_seen.csv')
val_unseen = pd.read_csv('../data/val_unseen.csv')
courses = pd.read_csv('../data/courses.csv').fillna('')
users = pd.read_csv('../data/users.csv').fillna('')

In [25]:
def get_popular_course(df_list, weight = [1, 64, 64]):
    popular_course = {}
    for i, df in enumerate(df_list):
        for course in df.course_id:
            for c in course.split(' '):
                try:
                    popular_course[c] += weight[i]
                except:
                    popular_course[c] = weight[i]
    return popular_course
popular_course = get_popular_course([train, val_seen, val_unseen])
for x in list(set(courses['course_id'].to_list()).difference(set(popular_course.keys()))):
    popular_course[x] = 0
popular_course_list = sorted(list(popular_course.items()), key=lambda x:x[1], reverse=True)

In [26]:
courses_interests = {}
for c, i in zip(courses.course_id, courses['groups+subgroups']):
    courses_interests[c] = i.split(',')

In [27]:
users_interests = {}
for u, i in zip(users.user_id, users.interests):
    users_interests[u] = i.split(',')

In [28]:
popular_course_with_interest = {}
for pc in popular_course_list:
    popular_course_with_interest[pc[0]] = courses_interests[pc[0]]
popular_course_with_interest = list(popular_course_with_interest.items())

In [29]:
val_unseen

,user_id,course_id
0,612c1fcd560d8100069aa5ba,5f14aeabcad0d0afe5ea3898 60ac8e42992b7aadf9d019d8
1,558b954c6dec460f00111131,5fc5ed671be929537e5283bb 5fc5edae001c9102feab8...
2,591de47900f58c070078cd99,60aeac37bca91777bf5bb114
3,612e5ebd31e50c0006fc4611,5fc5ee1b08b74a6e3723abd2 60cb0a440dabda80019d5...
4,60c248c62757c16af6160429,60a49d7c62c6aea8d38e3903
...,...,...
11617,617eb7aff2c5f00006592046,6030c9cd99e14cc2401e66b9 5fc5ee1b08b74a6e3723a...
11618,613253cdd5d02a0007225fe4,5e8e9d3c5a140c3d1e1c5245
11619,617eb70db23194000781f7cf,61381a3f33a3960006df1eb3
11620,5c9f3af172a72f0021508a87,5ce3a183138a45002046859d


In [30]:
def get_bought_courses(df_list):
    bought_courses = {}
    for df in df_list:
        for user, courses in zip(df.user_id, df.course_id):
            try:
                bought_courses[user] = courses.split(' ')
            except:
                bought_courses[user] += courses.split(' ')
    return bought_courses
bought_courses = get_bought_courses([train, val_seen, val_unseen])     

In [31]:
with open('./submit_unseen_course.csv', 'w') as f:
    f.write('user_id,course_id\n')
    for user in test_unseen.user_id:
        self_interest = set(users_interests[user])
        if(len(self_interest) == 0):
            rec = popular_course_with_interest
        else:
            temp = [[] for x in range(len(self_interest) + 1)]
            for c in popular_course_with_interest:
                c_interest = set(c[1])
                temp[len(self_interest.intersection(c_interest))].append(c[0])
            rec = [item for sublist in temp[::-1] for item in sublist]
        rec2 = []
        if user in bought_courses.keys():
            for c in rec:
                if(c not in bought_courses[user]):
                    rec2.append(c)
        else:
            rec2 = rec

        f.write('{},{}\n'.format(user, ' '.join(rec2[:50])))